In [4]:
import re
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

## Importando os dados

In [33]:
df = pd.read_csv('data/data_itens_hospitalares.csv')
df

,id_registro,itens
0,314768,PROSTIGMINE 0 5MG/1ML INJ *
1,314768,FOSFATO DE CLINDAMICINA 600MG/4ML INJ
2,314768,ULTIVA 2MG INJ *
3,314768,NAUSEDRON 2MG/ML AMP. C/ 2ML INJ *
4,314768,DORMONID 5MG/5ML INJ *
...,...,...
14451,622208,KEFAZOL 1G INJ *
14452,622208,NAUSEDRON 2MG/ML AMP. C/ 4ML INJ *
14453,622208,NOVALGINA AMP 2ML*
14454,622208,BEXTRA 40MG INJ *


## Pré-Processamento

- Broca e fresa

In [34]:
df_broca_fresa = df[[re.search("BROCA|FRESA",a) is not None for a in df.itens]]
df_broca_fresa

,id_registro,itens
26,314768,BROCA CARBONO CONICADENTADA 1.6M MICRO DRILL R...
117,315301,FRESA CONICA ESTREITA REF: 03000089S SYNTHES
404,321064,BROCA DE DESGASTE
494,327726,FRESA ESFERICA DE DESBASTE
648,333978,FRESA TIPO FCL 70 0 MM
...,...,...
13621,600759,BROCA ESFERICA
13623,600759,BROCA CRONICA LONGA 1 2MMX 50MM
13680,603253,BROCA ESFERICA 5.0MMX70MM D - REF: BED02310101
14178,617097,BROCA CORTANTE 3.10X70 MM REF: H22010331070


In [36]:
list_broca_fresa = pd.DataFrame(df_broca_fresa.groupby('id_registro').apply(lambda group: group.itens.tolist())).reset_index()
list_broca_fresa.columns = ['id_registro', 'itens']

list_broca_fresa

,id_registro,itens
0,314768,[BROCA CARBONO CONICADENTADA 1.6M MICRO DRILL ...
1,315301,[FRESA CONICA ESTREITA REF: 03000089S SYNTHES]
2,321064,[BROCA DE DESGASTE]
3,327726,[FRESA ESFERICA DE DESBASTE]
4,333978,[FRESA TIPO FCL 70 0 MM]
5,339564,[FRESA CONICA ESTREITA REF: 03000089S SYNTHES...
6,347447,"[BROCA DE DESGASTE, BROCA LINDERMAM, FRESA ESF..."
7,355646,[FRESA CIRURGICA BRD55H 3 5 RAZEK ]
8,355996,[FRESA CONICA ESTREITA REF: 03000089S SYNTHES...
9,383326,"[BROCA LINDERMAM, BROCAS DIAMANTADAS RAZEK RE..."


- Todos os itens

In [38]:
itens_sinonimos = [
    "ANCHORFIX 1.7 TI. 2 SUTURAS USP 2-0 NÃO ABSOR.REF: 112001723",
    "ANCHORFIX 1.7MM TITANO 2 SUTURAS NAO ABS REF: 0112071-723",
    "ANCORA 1,9 TI 2 SUTURAS USP 2-0 C/ AGULHAS REF: 0112011-923",
    "ANCORA 1.9 TI.2 SUTURAS 2-0 COM AGULHAS SARTORI - 112011923",
    "ANCORA FLEXIVEL COM APLICADOR 1.4MM 2 FIOS REF: 114041423",
    "ANCORTEC COM AGULHA 1.7MM - REF: 10300617",
    "BROCA CIRURGICA ESFERICA 3.5X50MM - REF: BED01810101",
    "BROCA CRONICA LONGA 1,2MMX 50MM",
    "BROCA ESFERICA",
    "BROCA ESFERICA 4.0MMX70MM BED.0199-01-01",
    "BROCA ESFERICA 5.0MMX70MM D - REF: BED02310101",
    "BROCA LINDEMANN 2,1X70MM REF: BLI00860101",
    "FRESA CIRURGICA 2.1MM - REF: RAZ70H21",
    "FRESA CIRURGICA RAZEK (3,5) REF:BCR70H35",
    "KIT ARTROSCOPIA DE ATM HORTRON REF: 1400.001",
    "KIT ARTROSCOPIA DE BUCO SETORMED - REF: 038000000",
    "KIT CANULA   CIRURGIA ENDOSCOPICA DE ATM II-001  940160000",
    "KIT INTRUMENTAL ATM SETORMED REF ATM001",
    "LAMINA RECIPROCA : R-01.0035/0.4-1",
    "LAMINA ULTRASSONICA PARA OSTEOTOMIA 0.6MM - REF: 03370013",
    "LAMINAS PARA OSTEOTOMIA 0.6MM - REF: 3370012",
    "MONITORIZAÇÃO NEURO FISIOLOG INTRA OPERAT NEUROMEP8",
    "KIT MONITORIZAÇAO - REF: NS006201015",
    "KIT MONITORIZACAO DO NERVO FACIAL",
    "KIT PARA MONITORIZAÇAO INTRA NEUROFISIOLOGICA IOM",
    "KIT DE MONITORIZACAO ELETROFISIOLOGICO",
    "KIT ELETRODO  MONITORIZAÇAO DO NERVO FACIAL",
    "MINIPARAFUSO MMF AUTO DRIVE 2X8MM - REF: 2092008",
    "PARAPARAFUSO DE BLOQ. 2.0X7.0 MM - REF: 955207",
    "PONTEIRA ULTRASSONICA OT6 0.75MM - REF: 3370006",
    "PONTEIRA ULTRASSONICA OT7A - REF: 3370011",
    "PONTEIRA ULTRASSONICA OT8L 0.6MM - REF: 3370013",
    "PONTEIRA MICRODISSECTORA VOLMED",
    "PONTA ULTRASSONICA H-SG1 - REF: Z305151",
    "PONTEIRA ULTRASSONICA OT9",
    "SONDA URETRAL N§ 10 EM PAPEL GRAU CIRURGICO - EMBRAMED",
    "TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MALLINCKRODT",
    "BROCA LINDEMANN 2,1X70MM REF: BLI00860101",
    "BROCA DE DESGASTE REF BCND01020101",
    "INATIVO -BROCA LINDEMANN LONGA 2.1X50MM  BLI00840101",
    "BROCA CONICA LONGA 1,2MMX70MM REF BCND00860101",
    "BROCA ESFERICA",
    "BROCA ESFERICA 5.0MMX70MM D - REF: BED02310101",
    "BROCA LINDEMANN REF: BLI.0036.01-01",
    "BROCA BUD REF: BUA.0018.01-01",
    "BROCA BUD4,0MMX70MM REF BUD00040101",
    "BROCA CONICA   BCND01160101",
    "BROCA CONICA CURTA 2.1 X 70MM REF.: BCNA.0199.01-01",
    "BROCA CÔNICA LONGA 1.6X70MM REF: BCND.0118.01-01 BCND0118010",
    "BROCA CONICA REF:BCND00700101",
    "BROCA ESFERICA 3.1X70MM - REF: BED01670101",
    "INATIVO-BROCA CORTANTE 2.3X 70MM : H22010323070",
    "BROCA CIRURGICA 703 - REF: BCND01970101",
    "BROCA CIRURGICA ENT70H 4.5MM - REF: 731760001",
    "BROCA CONICA CURTA 1.6X70MM REF: BCNA.0102.01-01",
    "BROCA CONICA CURTA 2.1MM X 50 MM REF BCND0197.01",
    "BROCA CÔNICA CURTA 2.1X70MM REF: BCND.0199.01-01",
    "BROCA ESFERICA 2,0X60MM REF BED01010101",
    "BROCA ESFERICA 4,0MM X 50MM REF: BED.0197.01-01 BED01970101",
    "BROCA LINDEMANN CURTA 2.1X50MM - REF: BLI00680101",
    "FRESA CIRURGICA RAZEK (3,5) REF:BCR70H35",
    "BROCA CONICA CURTA 1.2 X 50MM A ENGATE J - BCNA00680109",
    "BROCA CONICA CURTA D 1.2X50MM - REF: BCND00680101",
    "BROCA DIAMANTADA 3,10X70MM H24-01.0331070",
    "BROCA ESFERICA 2,5X60MM REF BED01330101",
    "BROCA ESFERICA 4,0X90MM REF BED02010101",
    "BROCA ESFERICA 5.0X50MM A - REF: BEA02290109",
    "BROCA ESFERICA 6X80MM REF BED02480101",
    "BROCA LINDERMANN - REF: 03000113S",
    "FRESA CIRURGICA 1.8MM - REF: RAZ70H18",
    "FRESA CIRURGICA RAZ70H 1.8MM - REF: 732010001",
    "FRESA CIRURGICA RAZEK 3.1  REF: BRD95R31",
    "FRESA DE LINDERMANN - REF: BCL70H21",
    "FRESAS CIRURGICAS RAZEK 1.8MM - REF: BRD70H18",
    "INATIVO -BROCA LINDERMANN CURTA 2,1X50MM .: BLI.0068.01-09",
    "INATIVO-BROCA ESFERICA 0,1MM X 50MM : BED01650101",
    "DISSECTOR ENGIMPLAN RETO 45X3MM E - LARANJA REF: 10459146",
    "DISSECTOR RETO 62X3MM E LARANJA - REF: 10459148",
    "DISSECTOR RETO PONTA COLORADO 52X3MM - REF: 10459147",
    "PONTEIRA MICRODISSECTORA VOLMED",
    "LAMINA RECIPROCANTE ARQUEADA 27X6.3X0.4MM REF  R010008041",
    "LAMINA RECIPROCANTE 22.5X6.3X0.4X53MM  - REF: R010037041",
    "LAMINA RECIPROCA 27X6,3X0,4MM REF:R010036/0,4-1",
    "INATIVO-LAMINA RECIPROCA : R-01.0035/0.4-1",
    "LAMINA RECIPRO. 22.5X6.3X0.4MM HASTE 43MM SK - R010009041",
    "LAMINA RECIPROCA CORTE RAIO 25X7,5X0,4MM REF R-01.0019/0,4-1",
    "MICRO LAMINA SAGITAL REF: S-30.0581/0.4-1",
    "LAMINA OSCILANTE 20MM TU - REF: R200178041",
    "LAMINA OSCILANTE 7MM TU REF R200176041",
    "LAMINA RECIPROCA 18X6,3X0,4MM REF: R-20.0006/0,4-1",
    "LAMILA RECIPROCANTE - REF: R010010041",
    "LAMINA BASAL DIREITA - REF: SR011T",
    "LAMINA BASAL ESQUERDA - REF: SR013T",
    "LAMINA CIRURGICA SAGITAL 18.5X9X0.4MM REF S100070041",
    "LAMINA DE SERRA - REF: ZO7161",
    "LAMINA DESCARTAVEL REF.: S-30.0585/0,4-2 S300585042",
    "LAMINA OSCILANTE 10MM TU - REF: R200177041",
    "LAMINA OSCILANTE 7MM (TU) REF: R-200176/0,4-1",
    "LAMINA RECIPROCA 14.5X6.3X0.5MM HASTE 43MM SK   - R010039051",
    "LAMINA RECIPROCA 27X6X0,04 PONTA 3MM REF: R120137041",
    "LAMINA RECIPROCANTE 22.5X6.3X0.5MM - REF: R010052051",
    "LAMINA SERRA RECIPROCANTE - REF: Y900073",
    "MEMBRANA BICAMADA REABSORVIVEL TECIDULAR 25X25  308026",
    "MEMBRANA STRAUMANN JASON 20X30MM REF AW682030",
    "MINIPARAFUSO EM TITANIO M4 2X16MM - REF: 2022016",
    "INATIVO - MINIPARAFUSO EM TITANIO M4 2X6MM - REF: 2022006",
    "MINIPARAFUSO EM TITANIO M4 2X6MM - REF: 2022006",
    "PARAFUSO M4 EM TITANIO 1.6/2.0MM(ORC) 2022016",
    "MINIPARAFUSO MMF AUTO DRIVE 2X8MM - REF: 2092008",
    "INATIVO-PARAFUSO BUCO1.6 E 2.0 OSTEOMED",
    "PARAFUSO DE BLOQUEIO 2.0X5.0MM - REF: 2402005",
    "PARAPARAFUSO 2.0X6MM - REF: M514306",
    "INATIVO - MINIPARAFUSO MMF AUTO DRIVE 2X11MM - REF:",
    "MICRO PARAFUSO EM TITANIO M4 1.2X14MM - REF: 2041214",
    "MINI PARAFUSO VERSA TORXDRIVE TI 2.4MMX20.0MM - REF: 2232420",
    "MINIPARAFUSO EM TITANIO M4 2X8MM - REF: 2022008",
    "MINIPARAFUSO MMF AUTO DRIVE 2X11MM - REF: 2092011",
    "PARAFUSO AUTO PERFURANTE 2.0X11.0MM - REF: 924029",
    "PARAFUSO AUTORROSCANTE 2.0X06MM - REF: DT52006265",
    "PARAFUSO CROSS EM TITANIO 1.5X2.0MM (ORC) REF:M51",
    "PARAFUSO FACIAL 1.5X12.0MM - REF: F15A12",
    "PARAFUSO ORTHOGNATICO 1.9X5.0MM - REF: O19005",
    "MINI PLACA 34 FUROS - REF: 2101021",
    "MINI PLACA AVANÇO DE M4 2MM - REF: 2100041",
    "INATIVO-PLACA EM TITÂNIO MINI/MICRO1.6/2.0MM ORC M4",
    "PLACA EM TITANIO MINI/MICRO1.6/2.0MM ORC - 2100041",
    "INATIVO -INATIVO-MINI PLACA TITANEO 2.0MM",
    "INATIVO -MICROPLACA RETA 4 FUROS",
    "INATIVO -MINIPLACA BUCO OSTEOMED",
    "INATIVO-PLACA EM TITÂNIO MINI/MICRO1.5/2.0MM ORC M4",
    "MICRO PLACA L DIREITA MEDIA - REF: 80100600000",
    "MINI PLACA PARA MENTO CHIN 4MM - REF: PA01030962",
    "MINI PLACA PAULUS 10MM SISTEMA 2.0 ORTOGNATICA - 3005220182",
    "MINI PLACA RETA 08 FUROS - REF: 10223008",
    "MINI PLACA RETA 4 FUROS - REF: PA01030014",
    "PLACA CRANIAL 2.0MM RETA 04 FUROS - REF: M4318",
    "PLACA CRANIANA RETA 2.0 23MM 4 FUROS REF  M4318",
    "PLACA FACIAL RETA LONGA 4F 0.5T FST305L04A",
    "PONTEIRA ULTRASSONICA OT12D REF:OT12S",
    "PONTEIRA ULTRASSONICA PIEZO REF: 03510004",
    "PONTA P/ OSTEOPLATIA US OT7  REF: 3370007",
    "PONTEIRA ULTRASSONICA OT8L 0.6MM - REF: 3370013",
    "PONTA P/OSTEOPLATIA US QT12 REF3370020",
    "LAMINA ULTRASSONICA PARA OSTEOTOMIA 0.6MM - REF: 03370013",
    "LAMINAS PARA OSTEOTOMIA 0.6MM - REF: 3370012",
    "PONTA DE ULTRA SOM VARIOSURG SG8 - REF: Z305109",
    "PONTEIRA ULTRASSONICA OT6 0.75MM - REF: 3370006",
    "PONTEIRA ULTRASSONICA PIEZO REF OT7S-3",
    "PONTA CIRURGICA ULTRA SONICA REF: US12",
    "PONTA DE PIEZO - REF: PPUS2N",
    "PONTA P/ OESTEOPLASTIA OP2 U S REF: 3370002",
    "PONTA ULTRASSONICA MICRO-SERRA 0.75MM - OT7A",
    "PONTEIRA DISSECTORA RETA 110X3MM - REF: 881440400",
    "PONTEIRA ULTRASSONICA 3X2X10CM - REF: OT1",
    "LAMINA PARA OSTEOTOMIA 0.55MM - REF: 3370007",
    "LAMINA PARA OSTEOTOMIA LOT7 - REF: EK30010M",
    "LAMINA ULTRASSONICA ESFERICA 55MM - REF: OT5",
    "BROCA BUD 6.0X50MM - REF: BUD00300101",
    "BROCA CIRURGICA BED 50DX4.0MM - REF: BED01970109",
    "BROCA CONICA CURTA 1.6X70MM REF: BCNA.0102.01-01",
    "BROCA CONICA LONGA 1,2MMX70MM REF BCND00860101",
    "BROCA ESFERICA 2,0X60MM REF BED01010101",
    "BROCA ESFERICA 5.0MMX70MM D - REF: BED02310101",
    "FRESA CIRURGICA RAZ55H 2.1 - REF: 732000002",
    "FRESA CIRURGICA RAZ70H 1.5MM - REF: 732010000",
    "FRESA FRL70H 5M DESGASTE - REF: 730860000",
    "FRESA RAZ55H H55 D1.5 TROCONCONICA - REF: 732000000",
    "PONTEIRA ULTRASSONICA OT12D REF:OT12S",
    "PONTEIRA ULTRASSONICA OT8L 0.6MM - REF: 3370013",
    "PONTA P/ OSTEOPLATIA US OT7  REF: 3370007",
    "PONTEIRA ULTRASSONICA OT7A - REF: 3370011",
    "PONTA DE ULTRA SOM VARIOSURG SG8 - REF: Z305109",
    "PLACA EM L 4 FUROS DIREITA - REF: 5508230",
    "PLACA EM L 4 FUROS ESQUERDA - REF: 5508231",
    "PLACA PARA CHIN BARRA 4MM - REF: 5509104",
    "PLACA DE PAULLUS (MENTO) SISTEMA UNIVERSAL ",
    "PLACA TITANIO SISTEMA 2.0 DE FIXACAO E RECONS ",
    "PLACA LEFORT PRE-DOBRADA 6MM 12 FUROS DIREITA  REF: B0884106",
    "PLACA LEFORT PRE-DOBRADA 6MM 12 FUROS ESQUERDA - B0884006",
    "MINI PLACA 34 FUROS - REF: 2101021",
    "MINI PLACA 6 FUROS - REF: 5508206",
    "MINI PLACA AVANÇO DE M4 2MM - REF: 2100041",
    "BROCA CONICA CURTA 1.6X50MM REF BCND01000109",
    "BROCA CONICA CURTA 2.1MM X 50 MM REF BCND0197.01",
    "BROCA CONICA REF:BCND00700101",
    "BROCA CORTE OBLIQUO 1.2MM PARA MICRO DRILL - REF: 277010212",
    "BROCA DE CARBONO 1.00MM MICRO DRILL REF 0277010210",
    "BROCA DE DESGASTE 2.2MM P/ MICRO DRILL STRYKER REF:1608-2-43",
    "BROCA DIAMANTADA - REF: 1608006099",
    "BROCA DIAMANTADA 4.0MM - REF: 5820012040",
    "BROCA DIAMANTADA ESFERICA 5.0X50 MM REF:BED02290201",
    "BROCA DIAMANTADA PARA NEURO 2.5MM ELITE DRILL REF 5820107025",
    "BROCA ELITE REDONDA DIAMANTADA 5.0MM - REF: 5820012050",
    "BROCA LINDERMAN DISSECÇÃO 2.1MM - REF: BCL55H21",
    "BROCA OVAL 4MM REF 0277010062",
    "BROCA REDONDA 2.0M P/ MICRO DRILL REF: 1608002043",
    "BROCA REDONDA 4.0MM P/ MICRO DRILL DESGASTE REF 0277010240",
    "BROCA REDONDA DIAMANTADA 3.5MM REF:1608006093",
    "BROCA REDONDA DIAMANTADA 4MM PARA MICRO DRILL REF 1608006095",
    "FRESA CIRURGICA - REF: BRD55H6.0",
    "FRESA CIRURGICA 1.5X70MM - REF: RAZ70H15",
    "FRESA CIRURGICA RAZEK (3,5) REF:BCR70H35",
    "DISSECTOR RETO 52X3MM - REF: PA02030007",
    "ENXERTO OSSEO BIOS-OSS COLLAGEN REF: 30585-7",
    "ENXERTO EM BLOCO BIO OSS 10X10X20MM - REF: 306022",
    "BIO-OSS COLLAGEN 100MG - REF: 305834",
    "HEMOSTATICO ABSORVIVEL BLEED STP 3GR - REF: 460020001",
    "LAMINA RECIPROCANTE ARQUEADA 27X6.3X0.4MM REF  R010008041",
    "LAMINA RECIPROCANTE 22.5X6.3X0.4X53MM  - REF: R010037041",
    "LAMINA RECIPROCANTE FINA ESTENDIDA 22.5MM - REF: 5100337133",
    "LAMINA RECIPROCANTE 27.0MM - REF: 5100337233",
    "LAMINA OSCILANTE 7MM TU REF R200176041",
    "LAMINA DE DESGASTE OVAL 4MM REF 0277010062",
    "LAMINA DE SERRA 27 6.42.9 0.6 TRAPEZOIDAL PSE REF 03000327S",
    "LAMINA RECIPROCANTE 22.5MM - REF: 5100337134",
    "LAMINA RECIPROCANTE 27X6.3X0.4MM - REF: R010008042",
    "LAMINAS PARA MICRO SERRA 4MM - REF: SR003T40",
    "MINIPARAFUSO EM TITANIO M4 2X6MM - REF: 2022006",
    "MINIPARAFUSO EM TITANIO M4 2X16MM - REF: 2022016",
    "MINIPARAFUSO EM TITANIO M4 2X6MM - REF: 2022006",
    "PARAFUSO AUTO ROSQUEANTE 2.0X5MM CROSS PIN - REF: 5020705",
    "PARAFUSO AUTPERFURANTE 1.6X8MM - REF: S1620008",
    "PARAFUSO EM TITANIO  UNIVERSAL 2.0 DE FIXACAO",
    "PARAFUSO IMF OSSEO EM CRUZ SELF DRILLING 1.3X10MM - S1991010"
]

In [39]:
itens = df[df['itens'].isin(itens_sinonimos)]

In [42]:
df_list = pd.DataFrame(itens.groupby('id_registro').apply(lambda group: group.itens.tolist())).reset_index()
df_list.columns = ['id_registro', 'itens']
df_list

,id_registro,itens
0,314768,[TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...
1,315301,[TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...
2,316737,[TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...
3,333978,[TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...
4,339564,[TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...
...,...,...
137,614109,[MINIPARAFUSO MMF AUTO DRIVE 2X8MM - REF: 2092...
138,614966,[KIT ARTROSCOPIA DE ATM HORTRON REF: 1400.001]
139,617097,[PONTA DE PIEZO - REF: PPUS2N]
140,621233,[KIT ARTROSCOPIA DE ATM HORTRON REF: 1400.001]


# Fp Growth

In [43]:
te = TransactionEncoder()

### Para Broca e Fresa

In [46]:
te_array = te.fit(list_broca_fresa['itens']).transform(list_broca_fresa['itens'])

broca_fresa = pd.DataFrame(te_array, columns = te.columns_)

broca_fresa.set_index(list_broca_fresa['id_registro'], inplace = True)

broca_fresa

,BROCA CARBIDE TUNGSTENIO DESGASTE CURTA 700 - H33104010,BROCA CARBONO CONICADENTADA 1.6M MICRO DRILL REF:277-010-216,BROCA CIRURGICA 703 - REF: BCND01970101,BROCA CIRURGICA CORTANTE AFUNILADA 1.2MM - REF: S70151,BROCA CIRURGICA ESFERICA 3.5X50MM - REF: BED01810101,BROCA CONICA CURTA 1 2X70MM REF:BCND00700101,BROCA CONICA CURTA 1.6X70MM REF: BCNA.0102.01-01,BROCA CONICA CURTA 2.1MM X 50 MM REF BCND0197.01,BROCA CONICA CURTA D 1.8X70MM - REF: BCND01340101,BROCA CORTANTE 3.10X70 MM REF: H22010331070,...,FRESA CONI ESTRECHA M 2.3 L CABEZA 5.2 - REF: 03000089S,FRESA CONICA ESTREITA REF: 03000089S SYNTHES,FRESA CORTANTE 3.1MM - REF: BCR70H31,FRESA EM TUNGSTENIO FORMATO CONICO ARRENDODADO - PM79,FRESA ESFERICA DE DESBASTE,FRESA OVAL 4.0 MM X 8.0 MM BURR EGG SHAPED REF 03000071S,FRESA PERA 40MM - REF: BRD70H40,FRESA PERIFORME 5.0 REF 03000077S,FRESA PIRIFORME M 4 C-CABECA 6.9 - REF: 03000237S,FRESA TIPO FCL 70 0 MM
id_registro,,,,,,,,,,,,,,,,,,,,,
314768,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
315301,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
321064,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
327726,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
333978,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
339564,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,False
347447,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
355646,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
355996,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False


In [47]:
frequent_itemsets_broca_fresa = fpgrowth(broca_fresa, min_support = 0.01, use_colnames = True)
frequent_itemsets_broca_fresa

,support,itemsets
0,0.018182,(BROCA CARBONO CONICADENTADA 1.6M MICRO DRILL ...
1,0.054545,(FRESA CONICA ESTREITA REF: 03000089S SYNTHES)
2,0.054545,(BROCA DE DESGASTE)
3,0.036364,(FRESA ESFERICA DE DESBASTE)
4,0.036364,(FRESA TIPO FCL 70 0 MM)
...,...,...
98,0.018182,"(BROCA LINDEMANN 2 1X70MM REF: BLI00860101, BR..."
99,0.018182,"(BROCA ESFERICA, BROCA CRONICA LONGA 1 2MMX 50MM)"
100,0.018182,"(BROCA LINDEMANN 2 1X70MM REF: BLI00860101, BR..."
101,0.018182,"(BROCA LINDEMANN 2 1X70MM REF: BLI00860101, BR..."


In [48]:
rules_broca_fresa = association_rules(frequent_itemsets_broca_fresa, metric="lift", min_threshold=-50)
rules_broca_fresa

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(BROCA DE DESGASTE),(BROCA LINDERMAM),0.054545,0.054545,0.018182,0.333333,6.111111,0.015207,1.418182
1,(BROCA LINDERMAM),(BROCA DE DESGASTE),0.054545,0.054545,0.018182,0.333333,6.111111,0.015207,1.418182
2,(FRESA ESFERICA DE DESBASTE),(FRESA TIPO FCL 70 0 MM),0.036364,0.036364,0.018182,0.500000,13.750000,0.016860,1.927273
3,(FRESA TIPO FCL 70 0 MM),(FRESA ESFERICA DE DESBASTE),0.036364,0.036364,0.018182,0.500000,13.750000,0.016860,1.927273
4,(BROCA DE DESGASTE),(FRESA ESFERICA DE DESBASTE),0.054545,0.036364,0.018182,0.333333,9.166667,0.016198,1.445455
...,...,...,...,...,...,...,...,...,...
131,(BROCA LINDEMANN 2 1X70MM REF: BLI00860101),"(BROCA ESFERICA, BROCA CRONICA LONGA 1 2MMX 50MM)",0.036364,0.018182,0.018182,0.500000,27.500000,0.017521,1.963636
132,(BROCA ESFERICA),"(BROCA LINDEMANN 2 1X70MM REF: BLI00860101, BR...",0.018182,0.018182,0.018182,1.000000,55.000000,0.017851,inf
133,(BROCA CRONICA LONGA 1 2MMX 50MM),"(BROCA LINDEMANN 2 1X70MM REF: BLI00860101, BR...",0.018182,0.018182,0.018182,1.000000,55.000000,0.017851,inf
134,(BROCA DIAMANTADA 2.30X70MM REF: H24-01.0323070),(BROCA CORTANTE 3.10X70 MM REF: H22010331070),0.018182,0.018182,0.018182,1.000000,55.000000,0.017851,inf


- lift maior que 1 

In [49]:
rules_broca_fresa[rules_broca_fresa.lift > 1]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(BROCA DE DESGASTE),(BROCA LINDERMAM),0.054545,0.054545,0.018182,0.333333,6.111111,0.015207,1.418182
1,(BROCA LINDERMAM),(BROCA DE DESGASTE),0.054545,0.054545,0.018182,0.333333,6.111111,0.015207,1.418182
2,(FRESA ESFERICA DE DESBASTE),(FRESA TIPO FCL 70 0 MM),0.036364,0.036364,0.018182,0.500000,13.750000,0.016860,1.927273
3,(FRESA TIPO FCL 70 0 MM),(FRESA ESFERICA DE DESBASTE),0.036364,0.036364,0.018182,0.500000,13.750000,0.016860,1.927273
4,(BROCA DE DESGASTE),(FRESA ESFERICA DE DESBASTE),0.054545,0.036364,0.018182,0.333333,9.166667,0.016198,1.445455
...,...,...,...,...,...,...,...,...,...
131,(BROCA LINDEMANN 2 1X70MM REF: BLI00860101),"(BROCA ESFERICA, BROCA CRONICA LONGA 1 2MMX 50MM)",0.036364,0.018182,0.018182,0.500000,27.500000,0.017521,1.963636
132,(BROCA ESFERICA),"(BROCA LINDEMANN 2 1X70MM REF: BLI00860101, BR...",0.018182,0.018182,0.018182,1.000000,55.000000,0.017851,inf
133,(BROCA CRONICA LONGA 1 2MMX 50MM),"(BROCA LINDEMANN 2 1X70MM REF: BLI00860101, BR...",0.018182,0.018182,0.018182,1.000000,55.000000,0.017851,inf
134,(BROCA DIAMANTADA 2.30X70MM REF: H24-01.0323070),(BROCA CORTANTE 3.10X70 MM REF: H22010331070),0.018182,0.018182,0.018182,1.000000,55.000000,0.017851,inf


In [50]:
rules_broca_fresa.lift.unique()

array([ 6.11111111, 13.75      ,  9.16666667,  9.16666667, 27.5       ,
       18.33333333, 18.33333333, 55.        ,  6.875     ])

- lift menor que 1 

In [51]:
rules_broca_fresa[rules_broca_fresa.lift < 1]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


# Para todos os itens

In [53]:
te_array = te.fit(df_list['itens']).transform(df_list['itens'])
df = pd.DataFrame(te_array, columns = te.columns_)
df.set_index(df_list['id_registro'], inplace = True)

In [54]:
frequent_itemsets = fpgrowth(df, min_support = 0.01, use_colnames = True)
frequent_itemsets

,support,itemsets
0,0.197183,(TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...
1,0.014085,(FRESA CIRURGICA 1.5X70MM - REF: RAZ70H15)
2,0.035211,(LAMINAS PARA MICRO SERRA 4MM - REF: SR003T40)
3,0.126761,(KIT CANULA CIRURGIA ENDOSCOPICA DE ATM II-0...
4,0.084507,(ANCORA 1.9 TI.2 SUTURAS 2-0 COM AGULHAS SARTO...
...,...,...
115,0.014085,(ANCHORFIX 1.7MM TITANO 2 SUTURAS NAO ABS REF:...
116,0.014085,(DISSECTOR ENGIMPLAN RETO 45X3MM E - LARANJA R...
117,0.014085,(DISSECTOR ENGIMPLAN RETO 45X3MM E - LARANJA R...
118,0.014085,(DISSECTOR ENGIMPLAN RETO 45X3MM E - LARANJA R...


In [55]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=-50)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(LAMINAS PARA MICRO SERRA 4MM - REF: SR003T40),(PONTA DE ULTRA SOM VARIOSURG SG8 - REF: Z305109),0.035211,0.098592,0.028169,0.800000,8.114286,0.024697,4.507042
1,(PONTA DE ULTRA SOM VARIOSURG SG8 - REF: Z305109),(LAMINAS PARA MICRO SERRA 4MM - REF: SR003T40),0.098592,0.035211,0.028169,0.285714,8.114286,0.024697,1.350704
2,(LAMINAS PARA MICRO SERRA 4MM - REF: SR003T40),(TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...,0.035211,0.197183,0.014085,0.400000,2.028571,0.007141,1.338028
3,(TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...,(LAMINAS PARA MICRO SERRA 4MM - REF: SR003T40),0.197183,0.035211,0.014085,0.071429,2.028571,0.007141,1.039003
4,(DISSECTOR RETO 52X3MM - REF: PA02030007),(LAMINAS PARA MICRO SERRA 4MM - REF: SR003T40),0.105634,0.035211,0.014085,0.133333,3.786667,0.010365,1.113218
...,...,...,...,...,...,...,...,...,...
287,(ANCHORFIX 1.7MM TITANO 2 SUTURAS NAO ABS REF:...,(DISSECTOR ENGIMPLAN RETO 45X3MM E - LARANJA R...,0.028169,0.014085,0.014085,0.500000,35.500000,0.013688,1.971831
288,(DISSECTOR ENGIMPLAN RETO 45X3MM E - LARANJA R...,(KIT MONITORIZACAO DO NERVO FACIAL),0.056338,0.028169,0.014085,0.250000,8.875000,0.012498,1.295775
289,(KIT MONITORIZACAO DO NERVO FACIAL),(DISSECTOR ENGIMPLAN RETO 45X3MM E - LARANJA R...,0.028169,0.056338,0.014085,0.500000,8.875000,0.012498,1.887324
290,(BROCA ESFERICA 5.0MMX70MM D - REF: BED02310101),(ANCORA FLEXIVEL COM APLICADOR 1.4MM 2 FIOS RE...,0.021127,0.049296,0.014085,0.666667,13.523810,0.013043,2.852113


- Lift Maior que 1

In [56]:
maior_um = rules[rules.lift > 1]
maior_um

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(LAMINAS PARA MICRO SERRA 4MM - REF: SR003T40),(PONTA DE ULTRA SOM VARIOSURG SG8 - REF: Z305109),0.035211,0.098592,0.028169,0.800000,8.114286,0.024697,4.507042
1,(PONTA DE ULTRA SOM VARIOSURG SG8 - REF: Z305109),(LAMINAS PARA MICRO SERRA 4MM - REF: SR003T40),0.098592,0.035211,0.028169,0.285714,8.114286,0.024697,1.350704
2,(LAMINAS PARA MICRO SERRA 4MM - REF: SR003T40),(TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...,0.035211,0.197183,0.014085,0.400000,2.028571,0.007141,1.338028
3,(TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...,(LAMINAS PARA MICRO SERRA 4MM - REF: SR003T40),0.197183,0.035211,0.014085,0.071429,2.028571,0.007141,1.039003
4,(DISSECTOR RETO 52X3MM - REF: PA02030007),(LAMINAS PARA MICRO SERRA 4MM - REF: SR003T40),0.105634,0.035211,0.014085,0.133333,3.786667,0.010365,1.113218
...,...,...,...,...,...,...,...,...,...
287,(ANCHORFIX 1.7MM TITANO 2 SUTURAS NAO ABS REF:...,(DISSECTOR ENGIMPLAN RETO 45X3MM E - LARANJA R...,0.028169,0.014085,0.014085,0.500000,35.500000,0.013688,1.971831
288,(DISSECTOR ENGIMPLAN RETO 45X3MM E - LARANJA R...,(KIT MONITORIZACAO DO NERVO FACIAL),0.056338,0.028169,0.014085,0.250000,8.875000,0.012498,1.295775
289,(KIT MONITORIZACAO DO NERVO FACIAL),(DISSECTOR ENGIMPLAN RETO 45X3MM E - LARANJA R...,0.028169,0.056338,0.014085,0.500000,8.875000,0.012498,1.887324
290,(BROCA ESFERICA 5.0MMX70MM D - REF: BED02310101),(ANCORA FLEXIVEL COM APLICADOR 1.4MM 2 FIOS RE...,0.021127,0.049296,0.014085,0.666667,13.523810,0.013043,2.852113


- Lift Menor que 1

In [57]:
menor_um = rules[rules.lift < 1]
menor_um

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
18,(KIT CANULA CIRURGIA ENDOSCOPICA DE ATM II-0...,(TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...,0.126761,0.197183,0.021127,0.166667,0.845238,-0.003868,0.963380
19,(TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...,(KIT CANULA CIRURGIA ENDOSCOPICA DE ATM II-0...,0.197183,0.126761,0.021127,0.107143,0.845238,-0.003868,0.978028
22,(TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...,(ANCORA 1.9 TI.2 SUTURAS 2-0 COM AGULHAS SARTO...,0.197183,0.084507,0.014085,0.071429,0.845238,-0.002579,0.985915
23,(ANCORA 1.9 TI.2 SUTURAS 2-0 COM AGULHAS SARTO...,(TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...,0.084507,0.197183,0.014085,0.166667,0.845238,-0.002579,0.963380
200,(DISSECTOR RETO 52X3MM - REF: PA02030007),(TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...,0.105634,0.197183,0.014085,0.133333,0.676190,-0.006745,0.926327
201,(TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 6.5 - MAL...,(DISSECTOR RETO 52X3MM - REF: PA02030007),0.197183,0.105634,0.014085,0.071429,0.676190,-0.006745,0.963164
